In [ ]:
# import matplotlib.pyplot as plt
import numpy as np
import pprint

import sys
import os

sys.path.append(os.path.abspath('..'))

from SafeRLBench import config
from SafeRLBench import Bench, BenchConfig, BestPerformance, Policy, ProbPolicy
from SafeRLBench.algo import PolicyGradient
from SafeRLBench.envs import LinearCar, GeneralMountainCar
from SafeRLBench.policy import LinearPolicy
    
np.set_printoptions(precision=5)

In [ ]:
config.logger_set_level()
config.logger_add_stream_handler()
# config.logger_add_file_handler('BenchTestLog.log')
config.monitor_set_verbosity(2)
config.jobs_set(1)

In [ ]:
# configure policy
policy = LinearPolicy(2, 1)

# configure environments
algs = [
    (PolicyGradient, [{'policy': policy, 'max_it': 1000, 'eps': 0.00001, 'estimator': 'central_fd'},
                      {'policy': policy, 'max_it': 1500, 'eps': 0.00001, 'estimator': 'central_fd'}])
]

env = [[
    (LinearCar, {'horizon': 200}),
    (LinearCar, {'horizon': 100})
]]


test_config = BenchConfig(algs, env)

In [ ]:
benchmark = Bench(test_config, [BestPerformance()])

In [ ]:
benchmark()

In [ ]:
benchmark.measures[0].result

In [ ]:
benchmark.runs